In [ ]:
!pip3 install ktrain

In [3]:
import numpy as np
import pandas as pd
import ktrain
from ktrain import text
import tensorflow as tf

We will use BERT for classification. It takes only positive values so we converted our labels from 0 to 10 rather than -4 to 4. Distil Bert is faster and almost 60% paramteres of BERT and achieves approximate accuracy to BERT.

In [ ]:
df = pd.read_csv('Stocks_Dataset.csv')
df.reset_index(drop=True, inplace=True)
df1 = pd.DataFrame
df1 = df[['Date', 'Headlines', 'News', 'sentiment']]
df1['News'] = df1.News.str.replace("(More)","")
df1['News1']=  df1["Headlines"].astype(str)+" "+df1["News"].astype(str) 
df1.drop('Headlines',axis=1, inplace=True)
df1.drop('News',axis=1,inplace=True)
df1.drop('Date',axis=1,inplace=True)
df1['sentiment']= df1['sentiment'].astype(int)
df1['sentiment'] = df1['sentiment']+4
df1.to_csv('train.csv')

In [29]:
df1.head()

,sentiment,News1
0,6,Aanchal Ispat: Outcome Of AGM × Outcome ...
1,1,"Aanchal Ispat's Board Meeting On August 14, 20..."
2,9,Aanchal Ispat Standalone September 2018 Net Sa...
3,3,Ashika Credit: Outcome Of AGM × The 24th...
4,5,"Ashika Credit's Board Meeting Held On July 27,..."


In [5]:
NUM_WORDS = 500000
MAXLEN = 512

trn, val, preproc = text.texts_from_csv('train.csv',
                     'News1',
                     preprocess_mode='distilbert',
                      label_columns = ['sentiment'],
                      maxlen=MAXLEN,
                      )

detected encoding: utf-8 (if wrong, set manually)
['sentiment_0', 'sentiment_1', 'sentiment_2', 'sentiment_3', 'sentiment_4', 'sentiment_5', 'sentiment_6', 'sentiment_7', 'sentiment_8']
      sentiment_0  sentiment_1  ...  sentiment_7  sentiment_8
4960          0.0          1.0  ...          0.0          0.0
613           0.0          0.0  ...          0.0          0.0
5865          0.0          0.0  ...          0.0          0.0
8220          0.0          0.0  ...          0.0          0.0
2594          0.0          0.0  ...          0.0          0.0

[5 rows x 9 columns]
['sentiment_0', 'sentiment_1', 'sentiment_2', 'sentiment_3', 'sentiment_4', 'sentiment_5', 'sentiment_6', 'sentiment_7', 'sentiment_8']
      sentiment_0  sentiment_1  ...  sentiment_7  sentiment_8
4268          0.0          0.0  ...          0.0          0.0
8353          0.0          0.0  ...          0.0          1.0
12            0.0          0.0  ...          0.0          0.0
1422          0.0          0.0  ... 

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 211
	95percentile : 557
	99percentile : 987


In [34]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) from keras_bert [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face transformers [https://arxiv.org/abs/1910.01108]


In [6]:
model = text.text_classifier('distilbert', train_data=trn, preproc=preproc)
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=10)

Is Multi-Label? False
maxlen is 512
done.


In [ ]:
learner.autofit(1e-3, 1)



begin training using triangular learning rate policy with max lr of 0.001...
 21/785 [..............................] - ETA: 6:53:01 - loss: 1.6781 - accuracy: 0.5429

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [43]:
len(trn)

7841